In [3]:
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import subprocess
import shutil
from selenium.webdriver.common.by import By
import pandas as pd
import time

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 셀레니움 신버전 크롬 드라이버 패키지

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.keys import Keys
# 진행률
from tqdm.notebook import tqdm
import re

ModuleNotFoundError: No module named 'webdriver_manager'

In [4]:
!pip install webdriver_manager

  Using cached webdriver_manager-3.8.6-py2.py3-none-any.whl (27 kB)
  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)


In [2]:
!pip install selenium

  Using cached selenium-4.9.1-py3-none-any.whl (6.6 MB)
  Using cached trio_websocket-0.10.2-py3-none-any.whl (17 kB)
  Using cached trio-0.22.0-py3-none-any.whl (384 kB)
  Using cached async_generator-1.10-py3-none-any.whl (18 kB)
  Using cached exceptiongroup-1.1.1-py3-none-any.whl (14 kB)
  Using cached outcome-1.2.0-py2.py3-none-any.whl (9.7 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)


In [67]:
class Crawling_rest_id:
    def __init__(self , sido , sigc , umdong):
        
        self.sido = sido
        self.sigc = sigc
        self.umdong = umdong
        self.frm = None
#         self.si_sub_address = None
#         self.DUML = None
        self.res_code = None
        
        options = Options()
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        
        self.restaurant = None
        self.address = None
        self.restaurant_count = None
        self.restaurant_name = None
        
        self.wait = WebDriverWait(self.driver, 10)
        
        
        print('__init__ pass')
    
    def load_data(self):
        print('load_data 진행중')
#         si_name = self.region
        if self.sido == '제주특별자치도' :
            df = pd.read_csv('제주도식당/jeju_rest_addcode.csv' , encoding='euc-kr')
        else :
            print('data_load 실패')
        # inpalce = True 원본객체에 바로 적용
        Data = df
        Data.dropna(inplace = True)
        Data = Data.query(f"시군구.str.contains('{self.sigc}') & 읍면동.str.contains('{self.umdong}')").reset_index(drop = True)
        self.frm = Data
        print('load_data 종료')
        # return si_data
    
    # 제주시 동읍면리 검색

#     def search_in_si(self) :
#         print('search_in_si 진행중')
#         df = pd.read_csv('제주도식당/제주도행정구역.csv' , encoding = 'euc-kr')
#         df_to_lst = df.행정구역.tolist()
#         si_data = self.si_data
# #         DUML = input('제주시의 동,읍,면,리를 입력하세요. : ')
#         DUML = '연동'
#         self.DUML = DUML
#         si_sub_address = si_data.query(f"도로명전체주소.str.contains('{DUML}')").reset_index(drop=True)
#         if DUML in df_to_lst :
#             self.si_sub_address = si_sub_address
#             # return si_sub_address
#         else :
#             print('해당 지역을 찾지 못하였습니다.')
#         print('search_in_si 종료')
    
    # 제주 제주시 영월읍 예시처럼 데이터 기입
    
    def add_columns(self) :
        print('add_columns 진행중')
        df = self.frm
        df['네이버주소추가'] = f'{self.sido[:2]} {self.sigc} {self.umdong}'
        self.frm = df
        print('add_columns 종료')

    def search(self) :
        try :
            self.load_data()
            self.add_columns()
        except :
            print('데이터 로드 중 오류발생')
            print('종료')
            
            
            
    def iframe_finder(self , iframe_id) :
        driver = self.driver
        wait = self.wait
        
        # iframe 초기화
        
        driver.switch_to.default_content()
        iframe = wait.until(EC.presence_of_element_located((By.ID, iframe_id)))
        driver.switch_to.frame(iframe)
    
    # 페이지 검색 후 결과가 1건 나왔을 때, 식닫을 클릭해서 이동해야하는 경우와 바로 식당 페이지가 나타나는 경우 구분
    def is_action_successful(self , driver):
        try:
            wait = WebDriverWait(driver, 3)
            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "N_KDL")))
            return True
        except:
            return False
        
    def search_simple_address(self , driver) :
        
        result = None
        
        time.sleep(0.3)
            
        wait = self.wait
        count = self.restaurant_count
        name = self.restaurant_name
        restaurant = self.restaurant
        address = self.address
        
        wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'hClKF')))
        rest_address_scarp = driver.find_elements(by=By.CLASS_NAME, value='hClKF')
        check_all_false = 0
        print('address = ', address)
        for num, add in enumerate(rest_address_scarp):
            if add.text == address:
                
                print(True)
                try :
                    wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'place_bluelink.YwYLL')))
                    driver.find_elements(by=By.CLASS_NAME, value='place_bluelink.YwYLL')[num].click()
                    time.sleep(0.3)
                except : 
                    try :
                        wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'place_bluelink.TYaxT')))
                        driver.find_elements(by=By.CLASS_NAME, value='place_bluelink.TYaxT')[num].click()
                        time.sleep(0.3)
                    except :
                        pass
                tmp = driver.current_url
                res_code = re.findall(r'place/(\d+)', tmp)
                print(f'{name} : {res_code[0]}')
                # url_lst.append((name , res_code[0]))
                result = (name , res_code[0])
                check_all_false += 1
                break                        
            else:
                print(False)
            time.sleep(0.5)
            # 버림
        if check_all_false == 0 :
            print(f"{name} : no_page")
            result = (name , '0000')
            # url_lst.append((name , '0000'))
            # result = self.search_all_address(driver)
        
        return result


        
    def search_all_address(self , driver) :
        
        check_all_false = 0
        
        wait = self.wait
        count = self.restaurant_count
        name = self.restaurant_name
        restaurant = self.restaurant
        address = self.address
        
        result = None
            
        for num , search_rest_name in enumerate(driver.find_elements(by = By.CLASS_NAME , value = 'N_KDL')) :
            # print(num)
            search_rest_name.click()
            
            self.iframe_finder('entryIframe')
            driver.implicitly_wait(2)
            wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'PkgBl')))
            
            driver.find_element(by = By.CLASS_NAME , value ='PkgBl').click()

            try : 
                wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'Y31Sf')))
            except :
                driver.find_element(by = By.CLASS_NAME , value ='PkgBl').click()
                
            time.sleep(0.5)
            search_address = driver.find_elements(by = By.CSS_SELECTOR , value = '.nQ7Lh')
            search_name = driver.find_element(by = By.CLASS_NAME , value = 'Fc1rA').text

            time.sleep(0.3)
            if address.split(' ')[0] + ' ' + address.split(' ')[1] in search_address[0].text[3:] :
                print('도, 시 일치')
                time.sleep(0.3)
                if address.split(' ')[2] in search_address[1].text[2:] :
                    print('세부 행정구역 일치')
                    time.sleep(0.3)
                    tmp = driver.current_url
                    res_code = re.findall(r'place/(\d+)', tmp)
                    print(f'{name} : {res_code[0]}')
                    # url_lst.append((name , res_code[0]))
                    result = (name , res_code[0])
                    check_all_false += 1
                    break
                else :
                    print('식당 주소 불일치')
                    print(f'확인중인 식당 이름 : {search_name} ')    
            else :
                print('식당 주소 불일치')
                print(f'확인중인 식당 이름 : {search_name} ')
                print(False)
            self.iframe_finder('searchIframe')

        if check_all_false == 0 :
            print(f"{name} : no_page")
            # url_lst.append((name , '0000'))
            result = (name , '0000')
        return result

        
    def find_url(self):
        df = self.frm
        driver = self.driver
        # options.add_argument('--headless')

        # driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        #number = int(input('크롤링을 시작할 사업장명 columns의 index를 입력(int) : '))
        
#         restaurant = df.사업장명.tolist()[number:]
        restaurant = df.사업장명.tolist()
        self.restaurant = restaurant
        
        address = df.네이버주소추가.unique().tolist()[0]
        self.address = address
        
        wait = self.wait

        
        url_lst = []
        
        for count , name in enumerate(restaurant) :
            
            self.restaurant_count = count
            self.restaurant_name = name
        
            url = f"https://map.naver.com/v5/search/{name}"
            driver.get(url)
            driver.implicitly_wait(5) 
            
            self.iframe_finder('searchIframe')
            is_successful = self.is_action_successful(driver)
            
            print(f'{count} : url 가져오기 시작')
            is_page = len(driver.find_elements(by=By.XPATH, value='//*[@id="_pcmap_list_scroll_container"]/ul/li'))
            time.sleep(0.5)
            if is_page > 1:
                if len(driver.find_elements(by=By.CLASS_NAME, value='hClKF')) != 0 :
                    url_lst.append(self.search_simple_address(driver))
                    
#                 elif len(driver.find_elements(by=By.CLASS_NAME, value='bVQnM')) != 0 :
                        
#                         time.sleep(0.3)
#                         wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'bVQnM')))
#                         rest_address_scarp = driver.find_elements(by=By.CLASS_NAME, value='bVQnM')
#                         for num, add in enumerate(rest_address_scarp):
#                             if add.text == address:
#                                 print(True)
#                                 wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'place_bluelink.TYaxT')))
#                                 driver.find_elements(by=By.CLASS_NAME, value='place_bluelink.TYaxT')[num].click()
#                                 time.sleep(0.3)
#                                 tmp = driver.current_url
#                                 res_code = re.findall(r'place/(\d+)', tmp)
#                                 print(f'{name} : {res_code[0]}')
#                                 url_lst.append((name , res_code[0]))
#                                 break                        
#                             else:
#                                 print(False)
#                             time.sleep(0.2)
                        
#                         if num+1 == len(rest_address_scarp) :
#                             print('all false')
#                             url_lst.append((name , '0000'))
                            
                else : 
                    url_lst.append(self.search_all_address(driver))
                        
            elif is_page == 1:
                time.sleep(0.3)
                if is_successful :
                    try : 
                        driver.find_element(by=By.CLASS_NAME , value = 'N_KDL').click()
                        self.iframe_finder('entryIframe')
                        tmp = driver.current_url
                        res_code = re.findall(r'place/(\d+)', tmp)
                        print(f'{name} : {res_code[0]}')
                    except : 
                        try :
                            time.sleep(1)
                            tmp = driver.current_url
                            res_code = re.findall(r'place/(\d+)', tmp)
                            print(f'{name} : {res_code[0]}')
                        except : 
                            print('로딩 오류')
                    finally :
                        url_lst.append((name, res_code[0]))
                else :
                    tmp = driver.current_url
                    res_code = re.findall(r'place/(\d+)', tmp)
                    print(f'{name} : {res_code[0]}')
                    url_lst.append((name , res_code[0]))
            else:
                print(f"{name} : no_page")
                url_lst.append((name , '0000'))
            print(f'{count} : url 가져오기 종료')
#             if count == 49 :
#                 break
        driver.quit()
        self.res_code = url_lst
        time.sleep(0.5)

    def Get_url(self) :
        self.search()
        self.find_url()
#         self.output_tocsv(self.res_code)
        return self.res_code

    def make_frm(self) :
        Data = self.frm
        name_lst = []
        id_lst = []
        for name , rest_id in self.res_code :
            name_lst.append(name)
            id_lst.append(rest_id)
        id_frm = pd.DataFrame({'사업장명' : name_lst , '식당id' : id_lst })
        
        # 결측치 제거
        id_frm = id_frm.query('id != "0000"')
        merged_frm = id_frm.merge(Data , on = '사업장명')
        
        if not os.path.exists(f'{self.sido}') :
            os.makedirs(f'{self.sido}') 
        else :
            pass
        if not os.path.exists(f'{self.sido}/{self.sigc}') :
            os.makedirs(f'{self.sido}/{self.sigc}') 
        else :
            pass
        if not os.path.exists(f'{self.sido}/{self.sigc}/id') :
            os.makedirs(f'{self.sido}/{self.sigc}/id') 
        else :
            pass
        
        merged_frm.to_csv(f'{self.sido}/{self.sigc}/{self.umdong}/id/{self.sido}_{self.sigc}_{self.umdong}_id.csv'
                   , index = False , encoding='euc-kr')
        return merged_frm
        

                    

In [68]:
crawling_id = Crawling_rest_id('제주특별자치도','제주시','연동')

__init__ pass


In [69]:
test_frm = crawling_id.Get_url()

load_data 진행중
load_data 종료
add_columns 진행중
add_columns 종료
0 : url 가져오기 시작
봉플라봉뱅 : 1713847517
0 : url 가져오기 종료
1 : url 가져오기 시작
address =  제주 제주시 연동
True
스톤아일랜드탭하우스 : 1244422132
1 : url 가져오기 종료
2 : url 가져오기 시작
address =  제주 제주시 연동
False
False
False
True
연화양꼬치 : 1482063535
2 : url 가져오기 종료
3 : url 가져오기 시작
address =  제주 제주시 연동
True
24시누름돌김치찌개 : 1028483996
3 : url 가져오기 종료
4 : url 가져오기 시작
address =  제주 제주시 연동
False
False
미연이네집 : no_page
4 : url 가져오기 종료
5 : url 가져오기 시작
address =  제주 제주시 연동
False
False
False
False
False
True
영화루 : 1109507832
5 : url 가져오기 종료
6 : url 가져오기 시작
성두길일오포차 : no_page
6 : url 가져오기 종료
7 : url 가져오기 시작
address =  제주 제주시 연동
False
False
False
False
False
False
False
False
True
앤드앤드 : 1308340997
7 : url 가져오기 종료
8 : url 가져오기 시작
address =  제주 제주시 연동
True
티엔미미 : 1982849089
8 : url 가져오기 종료
9 : url 가져오기 시작
address =  제주 제주시 연동
False
False
False
False
False
False
False
False
False
False
아하 : no_page
9 : url 가져오기 종료
10 : url 가져오기 시작
젠틀래빗제주연동점 : 1058918788
10 : url 가져오기 종료
11 : url 가져오기

96 : url 가져오기 시작
도, 시 일치
세부 행정구역 일치
순창갈비 : 1108531009
96 : url 가져오기 종료
97 : url 가져오기 시작
동이트는집신제주점 : 13444846
97 : url 가져오기 종료
98 : url 가져오기 시작
address =  제주 제주시 연동
True
중원(반점) : 32164019
98 : url 가져오기 종료
99 : url 가져오기 시작
address =  제주 제주시 연동
False
False
False
False
False
False
False
False
False
False
돈가 : no_page
99 : url 가져오기 종료
100 : url 가져오기 시작
라멘맛심 : 1218407521
100 : url 가져오기 종료
101 : url 가져오기 시작
향천회초밥 : 1842058271
101 : url 가져오기 종료
102 : url 가져오기 시작
(주)지앤엘(한식) : no_page
102 : url 가져오기 종료
103 : url 가져오기 시작
(주)세운상사(신한백화점구내) : no_page
103 : url 가져오기 종료
104 : url 가져오기 시작
address =  제주 제주시 연동
False
False
False
False
False
False
False
False
False
False
산호 : no_page
104 : url 가져오기 종료
105 : url 가져오기 시작
서영이네해돈가 : 1738684939
105 : url 가져오기 종료
106 : url 가져오기 시작
제주섬돈 : 1225985026
106 : url 가져오기 종료
107 : url 가져오기 시작
address =  제주 제주시 연동
True
만부정 : 13047263
107 : url 가져오기 종료
108 : url 가져오기 시작
오라버니회센타 : 16843094
108 : url 가져오기 종료
109 : url 가져오기 시작
빨간맛집제원점 : 37336341
109 : url 가져오기 종료
110 : url 가

False
False
False
False
False
False
True
차돌집 : 11660910
188 : url 가져오기 종료
189 : url 가져오기 시작
대관령 : no_page
189 : url 가져오기 종료
190 : url 가져오기 시작
삼해인관광호텔구내 : no_page
190 : url 가져오기 종료
191 : url 가져오기 시작
address =  제주 제주시 연동
True
베이스노트 : 37684308
191 : url 가져오기 종료
192 : url 가져오기 시작
슬로우식스 : no_page
192 : url 가져오기 종료
193 : url 가져오기 시작
포동정식도청점 : 19878395
193 : url 가져오기 종료
194 : url 가져오기 시작
address =  제주 제주시 연동
False
False
False
False
True
이쯔모 : 37657087
194 : url 가져오기 종료
195 : url 가져오기 시작
address =  제주 제주시 연동
False
False
False
False
False
False
False
False
False
False
여 : no_page
195 : url 가져오기 종료
196 : url 가져오기 시작
노가네전이랑국수 : no_page
196 : url 가져오기 종료
197 : url 가져오기 시작
address =  제주 제주시 연동
True
슈퍼보틀 : 1388612606
197 : url 가져오기 종료
198 : url 가져오기 시작
크라운호프연동점 : 1629358721
198 : url 가져오기 종료
199 : url 가져오기 시작
내가좋아하는것들 : 1810372609
199 : url 가져오기 종료
200 : url 가져오기 시작
address =  제주 제주시 연동
False
False
False
False
False
False
False
False
False
False
경성통닭 : no_page
200 : url 가져오기 종료
201 : url 가져오기 시작
add

280 : url 가져오기 시작
address =  제주 제주시 연동
True
전주토속청국장 : 20332741
280 : url 가져오기 종료
281 : url 가져오기 시작
빨간집연동점 : 32163973
281 : url 가져오기 종료
282 : url 가져오기 시작
준이자까야 : no_page
282 : url 가져오기 종료
283 : url 가져오기 시작
address =  제주 제주시 연동
False
True
두남자이야기 : 19786954
283 : url 가져오기 종료
284 : url 가져오기 시작
파크사이드호텔구내 : no_page
284 : url 가져오기 종료
285 : url 가져오기 시작
address =  제주 제주시 연동
True
풍천만가 : 37703784
285 : url 가져오기 종료
286 : url 가져오기 시작
address =  제주 제주시 연동
False
False
False
False
False
False
False
False
False
False
통영바다장어구이 : no_page
286 : url 가져오기 종료
287 : url 가져오기 시작
address =  제주 제주시 연동
False
True
소주방아지트 : 1978386688
287 : url 가져오기 종료
288 : url 가져오기 시작
모니카옛날통닭 : 16834930
288 : url 가져오기 종료
289 : url 가져오기 시작
address =  제주 제주시 연동
False
False
False
False
False
False
False
False
False
False
중화양꼬치 : no_page
289 : url 가져오기 종료
290 : url 가져오기 시작
스페이스비어펍 : no_page
290 : url 가져오기 종료
291 : url 가져오기 시작
address =  제주 제주시 연동
False
False
False
False
False
False
False
False
False
False
만한반점 : no_page
291 : url 가져오

address =  제주 제주시 연동
True
골목회수산 : 1198646965
384 : url 가져오기 종료
385 : url 가져오기 시작
킴스딜리버리 : 1166776009
385 : url 가져오기 종료
386 : url 가져오기 시작
address =  제주 제주시 연동
True
대굴대굴 : 33461057
386 : url 가져오기 종료
387 : url 가져오기 시작
address =  제주 제주시 연동
False
False
False
False
False
False
False
False
False
False
일 : no_page
387 : url 가져오기 종료
388 : url 가져오기 시작
address =  제주 제주시 연동
True
김밥상회 : 35065416
388 : url 가져오기 종료
389 : url 가져오기 시작
짱짱소주방 : 454743531
389 : url 가져오기 종료
390 : url 가져오기 시작
경희네몽생이 : no_page
390 : url 가져오기 종료
391 : url 가져오기 시작
address =  제주 제주시 연동
False
False
False
False
False
False
False
False
True
프로젝트064 : 86398282
391 : url 가져오기 종료
392 : url 가져오기 시작
담백한그집 : 19652551
392 : url 가져오기 종료
393 : url 가져오기 시작
address =  제주 제주시 연동
True
매옹이 : 1054949592
393 : url 가져오기 종료
394 : url 가져오기 시작
address =  제주 제주시 연동
False
False
False
False
False
False
False
False
False
False
물음표 : no_page
394 : url 가져오기 종료
395 : url 가져오기 시작
address =  제주 제주시 연동
True
보통김밥 : 1101729046
395 : url 가져오기 종료
396 : url 가져오기 시작

...

1070 : url 가져오기 시작
address =  제주 제주시 연동
False
False
False
False
False
False
False
False
False
False
안 : no_page
1070 : url 가져오기 종료
1071 : url 가져오기 시작
도도름한우와흑돈 : 1675391442
1071 : url 가져오기 종료
1072 : url 가져오기 시작
address =  제주 제주시 연동
False
False
False
False
False
False
False
False
False
False
야미 : no_page
1072 : url 가져오기 종료
1073 : url 가져오기 시작
address =  제주 제주시 연동
False
False
False
False
False
False
False
False
False
False
모이라 : no_page
1073 : url 가져오기 종료
1074 : url 가져오기 시작
address =  제주 제주시 연동
True
소못 : 1813957050
1074 : url 가져오기 종료
1075 : url 가져오기 시작
address =  제주 제주시 연동
False
False
False
False
False
False
False
False
False
False
에이플러스 : no_page
1075 : url 가져오기 종료
1076 : url 가져오기 시작
깅코 : 1442461925
1076 : url 가져오기 종료
1077 : url 가져오기 시작
address =  제주 제주시 연동
False
False
False
False
False
False
False
False
False
True
오늘김밥 : 1195651987
1077 : url 가져오기 종료
1078 : url 가져오기 시작
address =  제주 제주시 연동
False
False
False
False
False
False
False
False
False
False
최고집 : no_page
1078 : url 가져오기 종료
1079 :

False
False
True
선미정 : 1289232066
1167 : url 가져오기 종료
1168 : url 가져오기 시작
딸기가좋아신제주점 : no_page
1168 : url 가져오기 종료
1169 : url 가져오기 시작
정현이모 : 1218195599
1169 : url 가져오기 종료
1170 : url 가져오기 시작
address =  제주 제주시 연동
False
False
False
False
False
False
False
False
False
False
제이라운지 : no_page
1170 : url 가져오기 종료
1171 : url 가져오기 시작
피치랑신제주점 : 1095132484
1171 : url 가져오기 종료
1172 : url 가져오기 시작
address =  제주 제주시 연동
False
True
문송식당 : 1852599951
1172 : url 가져오기 종료
1173 : url 가져오기 시작
address =  제주 제주시 연동
True
예일갈비 : 17011574
1173 : url 가져오기 종료
1174 : url 가져오기 시작
address =  제주 제주시 연동
True
자매숯불 : 1316238694
1174 : url 가져오기 종료
1175 : url 가져오기 시작
망사리카페 : 1398892821
1175 : url 가져오기 종료
1176 : url 가져오기 시작
미보삿뽀로참치 : 1664227454
1176 : url 가져오기 종료
1177 : url 가져오기 시작
탑해장국앤돈가스 : no_page
1177 : url 가져오기 종료
1178 : url 가져오기 시작
address =  제주 제주시 연동
True
소랑분식 : 1269325809
1178 : url 가져오기 종료
1179 : url 가져오기 시작
로미파스타 : 1715382361
1179 : url 가져오기 종료
1180 : url 가져오기 시작
대영한밥상 : 1693801282
1180 : url 가져오기 종료
1181 : url 가져오기 시작
a

False
False
False
False
False
False
False
False
False
우솔 : no_page
1265 : url 가져오기 종료
1266 : url 가져오기 시작
청담회장님댁제주점 : 1565467780
1266 : url 가져오기 종료
1267 : url 가져오기 시작
핵밥제주연동점 : 1863678935
1267 : url 가져오기 종료
1268 : url 가져오기 시작
신광호프 : 1453918784
1268 : url 가져오기 종료
1269 : url 가져오기 시작
술빛정원 : 1011978142
1269 : url 가져오기 종료
1270 : url 가져오기 시작
신대장군집 : 1719406755
1270 : url 가져오기 종료
1271 : url 가져오기 시작
청담피자신제주점 : 1234168700
1271 : url 가져오기 종료
1272 : url 가져오기 시작
지엔디카페 : 1662734204
1272 : url 가져오기 종료
1273 : url 가져오기 시작
호야네꼬마김밥 : 1490488268
1273 : url 가져오기 종료
1274 : url 가져오기 시작
address =  제주 제주시 연동
False
True
장똘뱅이 : 1281852476
1274 : url 가져오기 종료
1275 : url 가져오기 시작
address =  제주 제주시 연동
False
False
False
False
False
False
False
False
False
False
동근이숯불두마리치킨 : no_page
1275 : url 가져오기 종료
1276 : url 가져오기 시작
임돈 : 1415878398
1276 : url 가져오기 종료
1277 : url 가져오기 시작
우정곱창연동점 : 1183365375
1277 : url 가져오기 종료
1278 : url 가져오기 시작
그시절동태촌신제주점 : 1057612716
1278 : url 가져오기 종료
1279 : url 가져오기 시작
연동가자스크린골프 : 1149996523
1279

1368 : url 가져오기 시작
델문도제주도립미술관점 : 1667500120
1368 : url 가져오기 종료
1369 : url 가져오기 시작
address =  제주 제주시 연동
False
False
False
False
False
False
False
False
False
False
마마 : no_page
1369 : url 가져오기 종료
1370 : url 가져오기 시작
쏭툰 : 1780864799
1370 : url 가져오기 종료
1371 : url 가져오기 시작
만땅피씨연동점 : no_page
1371 : url 가져오기 종료
1372 : url 가져오기 시작
아레나엑스피시방 : 1111294280
1372 : url 가져오기 종료
1373 : url 가져오기 시작
파스쿠찌제주한라병원점 : 1642097365
1373 : url 가져오기 종료
1374 : url 가져오기 시작
address =  제주 제주시 연동
False
True
피자굽는아저씨 : 20067954
1374 : url 가져오기 종료
1375 : url 가져오기 시작
대감님밥상앤덮밥집앤파스타비앤요맛나제주연동점 : no_page
1375 : url 가져오기 종료
1376 : url 가져오기 시작
농협중앙회북제주군지부구내 : no_page
1376 : url 가져오기 종료
1377 : url 가져오기 시작
플로베일배움터 : no_page
1377 : url 가져오기 종료
1378 : url 가져오기 시작
토방속에장독대 : 1650501767
1378 : url 가져오기 종료
1379 : url 가져오기 시작
address =  제주 제주시 연동
True
삐끄니끄 : 1005455511
1379 : url 가져오기 종료
1380 : url 가져오기 시작
엔제리너스한라수목원점 : 36948225
1380 : url 가져오기 종료
1381 : url 가져오기 시작
요거프레소바오젠점 : 37447485
1381 : url 가져오기 종료
1382 : url 가져오기 시작
쥬씨연동점 : no_pa

1474 : url 가져오기 시작
씨유제주남해오네뜨점 : 1890548274
1474 : url 가져오기 종료
1475 : url 가져오기 시작
디저트39제주연동대야점 : 1664044778
1475 : url 가져오기 종료
1476 : url 가져오기 시작
제주연동긱스타빽다방 : 1112844001
1476 : url 가져오기 종료
1477 : url 가져오기 시작
원더캣츠 : 1371757889
1477 : url 가져오기 종료
1478 : url 가져오기 시작
로얄호텔지하매점 : no_page
1478 : url 가져오기 종료
1479 : url 가져오기 시작
메가엠지씨커피제주롯데시티점 : no_page
1479 : url 가져오기 종료
1480 : url 가져오기 시작
address =  제주 제주시 연동
False
False
False
False
컴포즈커피연동점 : no_page
1480 : url 가져오기 종료
1481 : url 가져오기 시작
address =  제주 제주시 연동
False
False
False
False
False
False
False
False
True
빅브라더 : 1862225359
1481 : url 가져오기 종료
1482 : url 가져오기 시작
address =  제주 제주시 연동
False
False
False
False
False
False
False
False
False
False
보스턴수제버거 : no_page
1482 : url 가져오기 종료
1483 : url 가져오기 시작
봄봄카페오일장점 : 1883732255
1483 : url 가져오기 종료
1484 : url 가져오기 시작
빽다방제주연동사거리점 : 1570244855
1484 : url 가져오기 종료
1485 : url 가져오기 시작
줄리브신제주점 : 1360730509
1485 : url 가져오기 종료
1486 : url 가져오기 시작
노기다제주특별자치도의회점 : 1071812678
1486 : url 가져오기 종료
1487 : url 가져오기 시작
a